OOP PANDAS PROJECT

In [1]:
#importacao das bibliotecas a serem usadas
import pandas as pd
from datetime import datetime
from faker import Faker
from sqlalchemy import create_engine
import random
import string
import plotly.graph_objects as go
import re
fake = Faker()

In [3]:
# CLASSE ABSTRATA
from abc import ABC
class IMenssagens(ABC):
    def __init__(self, id_operador, id_contato, text, midia):
        pass
    def getIdOp(self):
        pass
    def getIdCont(self):
        pass
    def getDataCriacao(self):
        pass
    def getDataLida(self):
        pass
    def getText(self):
        pass
    def getMedia(self):
        pass
    def setText(self, new_text):
        pass
    def setMidia(self, new_midia):
        pass
    def citarText(self, msg):
        pass
    def citarMidia(self, msg): 
        pass
    def delText(self):
        pass
    def delMidia(self):
        pass

In [4]:
#CLASSE DE MENSAGENS
class Mensagens(IMenssagens):
    def __init__(self, id_operador, id_contato, text, midia):
        self.__id_operador = int(id_operador)
        self.__id_contato = int(id_contato)
        self.__text = str(text)
        self.__midia = str(midia)
        self.__data_criacao = datetime.today().strftime('%Y-%m-%d-%H:%M:%S')
        self.__data_lida = fake.date_time_between(start_date='now', end_date='+7d').strftime('%Y-%m-%d-%H:%M:%S')
    def getIdOp(self):
        return self.__id_operador
    def getIdCont(self):
        return self.__id_contato
    def getDataCriacao(self):
        return self.__data_criacao
    def getDataLida(self):
        return self.__data_lida
    def getText(self):
        return self.__text
    def getMedia(self):
        return self.__midia
    def setText(self, new_text):
        self._text = new_text
    def setMidia(self, new_midia):
        self._midia = new_midia
    def citarText(self, msg):
        return '"{}" {}'.format(self.__text, msg)
    def citarMidia(self, msg):
        return '"{}" {}'.format(self.__midia, msg)
    def delText(self):
        self.__text = None
    def delMidia(self):
        self.__midia = None

In [22]:
# CRIAR TABELA
mensagens = pd.DataFrame(columns=['id_operador','id_contato','texto','midia','data_criacao', 'data_lida'])

In [6]:
# POPULAR DF COM DADOS ALEATORIOS
def popular(amount):
    letras = string.ascii_lowercase
    for i in range(amount):
        obj = (Mensagens(random.randint(1,100), random.randint(1,1000), ''.join(random.choice(letras) for i in range(10)), random.choice([None, 'midia_generica'])))
        mensagens.loc[i] = [obj.getIdOp(), obj.getIdCont(), obj.getText(), obj.getMedia(), obj.getDataCriacao(), obj.getDataLida()]

In [7]:
# FUNCAO CRIAR O BANCO DE DADOS
def criar_db():
    engine = create_engine('sqlite:///:memory:')
    mensagens.to_sql('mensagens', engine)
    sql_msgs = pd.read_sql_table('mensagens', engine, columns = ['id_operador','id_contato','texto','midia','data_criacao', 'data_lida'])
    return(sql_msgs)

In [10]:
# FUNCAO DE ANALISE DA VOLUMETRIA
def analise_volumetrica(mensagens):
    #Total msgs
    msg_total = len(mensagens)
    #Total midias
    midias = len(mensagens.query('midia != "None"'))
    #Quantidade media de midias
    media_msg = midias/msg_total
    #Media de msgs enviadas por operador
    group = banco.groupby(['id_operador', 'texto']).size().reset_index(name='counts')
    sum = group.groupby('id_operador').sum()
    mean = sum.median()['counts']
    #Mediana de msgs enviadas por operador
    median = sum.mean()['counts']
    #Quantidade maxima de msgs enviadas por operador
    max = sum.max()['counts']
    #Quantidade minima de msgs enviadas por operador
    min = sum.min()['counts']
    #Moda das msgs enviadas por operador
    moda = sum.mode()['counts']
    #Porcentagem de msgs enviadas que sao somente texto
    diff = round((((msg_total - midias)/msg_total)*100),2)
    #Tempo medio de leitura das msgs
    data = banco.sort_values(by='data_lida')[['data_criacao', 'data_lida']]
    data = data.apply(pd.to_datetime)
    data = (data['data_lida'] - data['data_criacao'])
    media_tempo = round(data.mean().total_seconds(),2)
    if(media_tempo <= 60):
        media_tempo = "{} segundos".format(round(media_tempo,2))
    elif(media_tempo >60 and media_tempo <= 3600):
        media_tempo = "{} minutos".format(round(media_tempo/60,2))
    elif(media_tempo >3600 and media_tempo <= 86400):
        media_tempo = "{} horas".format(round(media_tempo/3600,2))
    else:
        media_tempo = "{} dias".format(round(media_tempo/86400,2))
    #Tempo maximo de leitura das msgs
    max_tempo = round(data.max().total_seconds(),2)
    if(max_tempo <= 60):
        max_tempo = "{} segundos".format(round(max_tempo,2))
    elif(max_tempo >60 and max_tempo <= 3600):
        max_tempo = "{} minutos".format(round(max_tempo/60,2))
    elif(max_tempo >3600 and max_tempo <= 86400):
        max_tempo = "{} horas".format(round(max_tempo/3600,2))
    else:
        max_tempo = "{} dias".format(round(max_tempo/86400,2))
    #Tempo minimo de leitura das msgs
    min_tempo = round(data.min().total_seconds(),2)
    if(min_tempo <= 60):
        min_tempo = "{} segundos".format(round(min_tempo,2))
    elif(min_tempo >60 and min_tempo <= 3600):
        min_tempo = "{} minutos".format(round(min_tempo/60,2))
    elif(min_tempo >3600 and min_tempo <= 86400):
        min_tempo = "{} horas".format(round(min_tempo/3600,2))
    else:
        min_tempo = "{} dias".format(round(min_tempo/86400,2))
    #Analise
    analise = {
        'total_msgs': msg_total,
        'total_midias': midias,
        'media_midias': round(media_msg,2),
        'media_msgs_op':mean,
        'mediana_msgs_op': median,
        'max_msgs_op': max,
        'min_msgs_op': min,
        'moda_msgs_op':moda[0],
        'porc_msgs_text':str(diff)+"%",
        'temp_medio_leit': media_tempo,
        'temp_max_leit':max_tempo,
        'temp_min_leit':min_tempo
    }
    return analise

In [11]:
def generate_graph(banco):
    figure = analise_volumetrica(banco)
    # GRAFICO DE PIZZA
    # Comparacao entre msgs total msgs e total midias
    labels = ['Texto', 'Midias']
    values = [(figure['total_msgs']-figure['total_midias']),figure['total_midias']]
    fig1 = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+percent', insidetextorientation='radial')])
    fig1.update_layout(title='<b> A partir dos dados selecionados e possivel concluir que: </b> <br> <br>  • Registra-se <b>{}</b> mensagens enviadas <br>  • A porcentagem de msgs de texto enviadas e de <b>{}</b> em relacao ao total <br>  • A relacao entre msgs de texto e midia e <b>{}</b> {}'.format(figure['total_msgs'],figure['porc_msgs_text'], str(round((((figure['total_msgs']-figure['total_midias'])-figure['total_midias'])*100/(figure['total_msgs']-figure['total_midias'])),3))+"%", "Maior" if (figure['total_msgs']-figure['total_midias']) > figure['total_midias'] else "Menor"))
    fig1.show()
    # GRAFICO DE BARRAS
    # Comparacao entre msgs total msgs 
    x = ['Media', 'Mediana', 'Minimo', 'Maximo', 'Moda']
    y = [figure['media_msgs_op'], figure['mediana_msgs_op'], figure['min_msgs_op'], figure['max_msgs_op'], figure['moda_msgs_op']]
    fig2 = go.Figure(
        data=[go.Bar(x = x,y = y)]
    )
    fig2.layout.template='seaborn'
    fig2.update_layout(title='Mensagens enviadas por operador',
                    yaxis_title='Quantidade de mensagens',
                    xaxis_title='<b> A partir dos dados selecionados e possivel concluir que: </b> <br> <br> •A quantidade minima de msgs enviadas por operador e <b>{}</b> {} que a media <br> • A quantidade maxima de msgs enviadas por operador e <b>{}</b> {} que a media<br> • A quantidade mais comum de msgs enviadas por operador e de <b>{}</b> msgs'.format(str(round(((figure['media_msgs_op']-figure['min_msgs_op'])*100/figure['media_msgs_op']),2))+"%", "Maior" if figure['min_msgs_op'] > figure['media_msgs_op'] else "Menor", str(round(((figure['max_msgs_op']-figure['media_msgs_op'])*100/figure['media_msgs_op']),2))+"%","Maior" if figure['max_msgs_op'] > figure['media_msgs_op'] else "Menor",figure['moda_msgs_op']))
    fig2.show()
    #GRAFICO DE LINHAS
    #Quantidade de msgs lidas por dia
    banco['data_lida'] = pd.to_datetime(banco['data_lida'], errors='coerce').dt.normalize()
    agrupamento = banco[['texto','data_lida']].groupby('data_lida').size().reset_index(name='counts')
    fig3 = go.Figure(data=go.Scatter(x=agrupamento['data_lida'], y=agrupamento['counts']))
    fig3.update_layout(title='Quantidade de mensagens lidas por dia',
                    xaxis_title='<b> A partir dos dados selecionados e possivel concluir que: </b> <br> <br> • O dia em que mais mensagens foram lidas foi <b>{}</b> totalizando <b>{}</b> mensagens <br> • O dia em que menos mensagens foram lidas foi <b>{}</b> totalizando <b>{}</b> mensagens'.format(agrupamento.query('counts == {}'.format(agrupamento['counts'].max()))['data_lida'].iloc[0].strftime('%d/%m'),agrupamento.max()['counts'], agrupamento.query('counts == {}'.format(agrupamento['counts'].min()))['data_lida'].iloc[0].strftime('%d/%m'), agrupamento.min()['counts']),

                    yaxis_title='Quantidade de mensagens')
    fig3.show()
    # GRAFICO DONUT
    #conversao de tempo pra minutos
    lista = [figure['temp_medio_leit'], figure['temp_max_leit'], figure['temp_min_leit']]
    time = []
    for i in lista:
        x = [int(s) for s in re.findall(r'\b\d+\b', i)]
        t = float(str(x[0])+'.'+str(x[1]))
        if('dias' in i):
            t = t*1440
        elif('horas' in i):
            t = t*60
        elif('segundos' in i):
            t = t/60
        else:
            t = t
        time.append(t)
    #grafico
    colors = ['gold', 'mediumturquoise', 'darkorange']
    fig4 = go.Figure(data=[go.Pie(labels=['Media (min)','Maximo (min)','Minimo (min)'],
                                values=[time[0],time[1],time[2]], hole = .3)])
    fig4.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                    marker=dict(colors=colors, line=dict(color='#000000', width=2)))
    fig4.update_layout(title='<b> A partir dos dados selecionados e possivel concluir que: </b> <br> <br> • Registra-se <b>{}</b> mensagens enviadas no total de <b>{}</b> dias <br> • O tempo minimo de leitura e de <b>{}</b> menor em relacao a media <br> • O tempo maximo de leitura e de <b>{}</b> maior em relacao a media'.format(figure['total_msgs'], len(agrupamento), str(round(((time[0]-time[2])*100/time[0]),4))+"%", str(round(((time[1]-time[0])*100/time[0]),2))+"%"))
    fig4.show()